In [ ]:
# Câu 2: Biến đổi ảnh theo menu F, L, H (FFT, Butterworth Lowpass/Highpass)
import numpy as np
import matplotlib.pyplot as plt
import imageio.v2 as iio
from PIL import Image
import math
import os
from scipy.fft import fft2, fftshift, ifft2

#Thư mục chứa ảnh
folder = "exercise"
output_folder = "output_cau2"
os.makedirs(output_folder, exist_ok=True)

def fast_fourier_transform(img):
    f = fft2(img)
    fshift = fftshift(f)
    magnitude_spectrum = 20 * np.log(np.abs(fshift) + 1)
    return magnitude_spectrum.astype(np.uint8)

def butterworth_lowpass(img, cutoff=30, order=2):
    rows, cols = img.shape
    crow, ccol = rows // 2 , cols // 2
    u = np.arange(rows)
    v = np.arange(cols)
    U, V = np.meshgrid(u - crow, v - ccol, indexing='ij')
    D = np.sqrt(U**2 + V**2)
    H = 1 / (1 + (D / cutoff)**(2*order))
    
    F = fft2(img)
    F_shift = fftshift(F)
    G = F_shift * H
    img_back = np.abs(ifft2(np.fft.ifftshift(G))) #type: ignore
    return np.clip(img_back, 0, 255).astype(np.uint8)

def butterworth_highpass(img, cutoff=30, order=2):
    rows, cols = img.shape
    crow, ccol = rows // 2 , cols // 2
    u = np.arange(rows)
    v = np.arange(cols)
    U, V = np.meshgrid(u - crow, v - ccol, indexing='ij')
    D = np.sqrt(U**2 + V**2)
    H = 1 / (1 + (cutoff / D)**(2*order))
    H[D == 0] = 0

    F = fft2(img)
    F_shift = fftshift(F)
    G = F_shift * H
    img_back = np.abs(ifft2(np.fft.ifftshift(G))) # type: ignore
    return np.clip(img_back, 0, 255).astype(np.uint8)

menu = {
    'F': fast_fourier_transform,
    'L': butterworth_lowpass,
    'H': butterworth_highpass
}

def list_images(folder):
    images = []
    for file in os.listdir(folder):
        if file.lower().endswith(('.jpg', '.png')):
            images.append(f"{folder}/{file}")
    return images

print("Chọn phương pháp: F (FFT), L (Butterworth Lowpass), H (Butterworth Highpass)")
choice = input("Nhập chữ tương ứng: ").upper()

if choice in menu:
    filenames = list_images(folder)
    for path in filenames:
        filename = path.split("/")[-1]
        img = Image.open(path).convert('L')
        img_np = np.array(img)
        transformed = menu[choice](img_np)

        plt.imshow(transformed, cmap='gray')
        plt.title(f"{filename} - {choice}")
        plt.axis('off')
        plt.show()

        out_path = f"{output_folder}/{filename.split('.')[0]}_{choice}.png"
        iio.imwrite(out_path, transformed)
else:
    print("Lựa chọn không hợp lệ!")
